In [1]:
from docplex.mp.model import Model

Neka je $x$ broj fiksnih a $y$ broj mobilnih telefona, nek vazi sledece:
$$ x \geq 100 $$
$$ y \geq 100 $$
$$ 0.4*x + 0.6*y \leq 3x_1 $$
$$ 0.5*x + 0.4*y 2 $$
Naci maksimalan profit ako je za svaki fiksni telefon profit 12 a za svaki mobilni 20.

In [2]:
model = Model(name = "phones")

In [3]:
x = model.continuous_var(lb=100, name='x')
y = model.continuous_var(lb=100, name='y')

In [4]:
model.add_constraint(0.4*x + 0.6*y <= 400)

docplex.mp.LinearConstraint[](0.400x+0.600y,LE,400)

In [5]:
model.add_constraint(0.5*x + 0.4*y <= 490)

docplex.mp.LinearConstraint[](0.500x+0.400y,LE,490)

In [6]:
model.print_information()

Model: phones
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 2
   - linear=2
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [7]:
model.maximize(12*x + 20*y)

In [8]:
model.solve()

docplex.mp.solution.SolveSolution(obj=13200,values={x:100,y:600})

In [9]:
model.print_solution()

objective: 13200.000
status: OPTIMAL_SOLUTION(2)
  x=100.000
  y=600.000


In [10]:
def read_line_of_ints(f):
    return [int(x) for x in f.readline().split()]
def read_instance(file_path: str):
    with open(file_path, 'r') as f:
        m, n = read_line_of_ints(f)
        cost = [read_line_of_ints(f) for _ in range(m)]
        facility_cost = read_line_of_ints(f)
        return cost, facility_cost

# UFLP

- skup korisnika I
- skup resursa J
- cena dodeljivanja korisnika i resursu j - $c_{ij}$
- cena uspostavljanja resursa j - $f_j$

$$min \sum_{i \in I} \sum_{j \in J} x_{ij} c_{ij}  + \sum_{j \in J} y_j f_j $$

$$ \sum_{j \in J} x_{ij} = 1, \forall i \in I $$

$$ x_{ij} \leq y_j, \forall i \in I, \forall j \in J $$

$$ x_{ij} \in \{0,1\} $$

$$ y_j \in \{0,1\} $$

In [11]:
cost, facility_cost = read_instance('uflp1.txt')
num_users = len(cost)
num_facilities = len(facility_cost)
print(num_users, num_facilities)

3 3


In [12]:
model = Model('uflp')

In [13]:
x = model.binary_var_matrix(num_users, num_facilities, name='x')

In [14]:
y = model.binary_var_list(num_facilities, name='y')

In [15]:
for i in range(num_users):
    model.add_constraint(sum(x[i,j] for j in range(num_facilities)) == 1)

In [16]:
for i in range(num_users):
    for j in range(num_facilities):
        model.add_constraint(x[i,j] <= y[j])

In [17]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [18]:
first_part = sum(y[j] * facility_cost[j] for j in range(num_facilities))
second_part = sum(sum(x[i,j] * cost[i][j] for j in range(num_facilities)) for i in range(num_users))
model.minimize(first_part + second_part)

In [19]:
solution = model.solve()
model.print_solution()

objective: 34
status: OPTIMAL_SOLUTION(2)
  x_0_0=1
  x_1_0=1
  x_2_0=1
  y_0=1


In [20]:
solution.display()

solution for: uflp
objective: 34
status: OPTIMAL_SOLUTION(2)
x_0_0 = 1
x_1_0 = 1
x_2_0 = 1
y_0 = 1


# SSCFLP

- skup korisnika I
- skup resursa J
- cena dodeljivanja korisnika i resursu j - $c_{ij}$
- cena uspostavljanja resursa j - $f_j$

dodatno:
- svaki korisnik i ima potraznju - $d_i$
- svaki resurs j ima kapacitet - $s_j$

$$min \sum_{i \in I} \sum_{j \in J} x_{ij} c_{ij}  + \sum_{j \in J} y_j f_j $$

$$ \sum_{j \in J} x_{ij} = 1, \forall i \in I $$

$$ \sum_{i \in I} d_i x_{ij} \leq s_j y_j, \forall j \in J $$

In [22]:
d = [3 for i in range(num_users)]
s = [5 for j in range(num_facilities)]

In [23]:
model = Model('sscflp')

In [25]:
x = model.binary_var_matrix(num_users, num_facilities, name='x')
y = model.binary_var_list(num_facilities, name='y')

In [26]:
for i in range(num_users):
    model.add_constraint(sum(x[i,j] for j in range(num_facilities)) == 1)

In [28]:
for j in range(num_facilities):
    model.add_constraint(sum(x[i,j]*d[i] for i in range(num_users)) <= y[j]*s[j])

In [30]:
first_part = sum(y[j] * facility_cost [j] for j in range(num_facilities))

In [31]:
second_part = sum(sum(x[i,j] * cost[i][j] for j in range(num_facilities)) for i in range(num_users))

In [32]:
model.minimize(first_part + second_part)
model.print_information()

Model: sscflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [38]:
model.solve()
model.print_solution()

objective: 51
status: OPTIMAL_SOLUTION(2)
  x_0_0=1
  x_1_1=1
  x_2_2=1
  y_0=1
  y_1=1
  y_2=1


# MSCFLP

$$min \sum_{i \in I} \sum_{j \in J} x_{ij} c_{ij}  + \sum_{j \in J} y_j f_j $$

$$ \sum_{j \in J} x_{ij} = 1, \forall i \in I $$

$$ \sum_{i \in I} d_i x_{ij} \leq s_j y_j, \forall j \in J $$

In [39]:
model = Model('mscflp')

In [40]:
x = model.continuous_var_matrix(num_users, num_facilities, name='x', lb=0, ub=1)
y = model.binary_var_list(num_facilities, name='y')

In [41]:
for j in range(num_facilities):
    model.add_constraint(sum(x[i,j]*d[i] for i in range(num_users)) <= y[j] * s[j])

In [42]:
for i in range(num_users):
    model.add_constraint(sum(x[i,j] for j in range(num_facilities)) == 1)

In [43]:
first_part = sum(y[j] * facility_cost [j] for j in range(num_facilities))
second_part = sum(sum(x[i,j] * cost[i][j] for j in range(num_facilities)) for i in range(num_users))

In [44]:
model.minimize(first_part + second_part)
model.print_information()

Model: mscflp
 - number of variables: 12
   - binary=3, integer=0, continuous=9
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [45]:
model.solve()
model.print_solution()

objective: 35.667
status: OPTIMAL_SOLUTION(2)
  x_0_0=0.667
  x_0_2=0.333
  x_1_0=1.000
  x_2_2=1.000
  y_0=1
  y_2=1


In [46]:
import numpy as np
from scipy.optimize import linprog

Data je funkcija $f$:
$$f(x_1, x_2) = 3x_1 + 4x_2$$

Pronaci minimum funkcije $f$ ako za nepoznate $x_1$ i $x_2$ vazi:
$$ x_1 + 2x_2 \leq 14 $$
$$ x_2 \leq 3x_1 $$
$$ x_1 - x_2 \leq 2 $$

In [48]:
c = np.array([3,4])

A_ub = np.array([[1,2],
                 [-3,1],
                 [1,-1]])

b_ub = np.array([14,0,2])

linprog(c, A_ub, b_ub)

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [ 0.000e+00  0.000e+00]
           nit: 0
         lower:  residual: [ 0.000e+00  0.000e+00]
                marginals: [ 3.000e+00  4.000e+00]
         upper:  residual: [       inf        inf]
                marginals: [ 0.000e+00  0.000e+00]
         eqlin:  residual: []
                marginals: []
       ineqlin:  residual: [ 1.400e+01  0.000e+00  2.000e+00]
                marginals: [-0.000e+00 -0.000e+00 -0.000e+00]

In [49]:
from scipy.optimize import minimize

Data je funkcija $f$:
$$ f(x, y) = 2xy + 2x - x^2 - y^2 $$

Data su ogranicenja:$$ g_1(x, y) = x ^3 - y = 0 $$
$$ g_2(x, y) = y - 1 >= 0 $$

In [51]:
def f(x):
    return 2*x[0]*x[1] + 2*x[0] - x[0]**2 - x[1]**2
c = [
    {
        'type': 'eq',
        'fun': lambda x: x[0]**3 - x[1],
        'jac': lambda x: np.array([3*x[0]**2, -1])
    },
    {
        'type': 'ineq',
        'fun': lambda x: x[1] - 1,
        'jac': lambda x: np.array([0, 1])
    }
]
minimize(f, x0=(2,4), constraints=c, method='SLSQP')

 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: -1.2343347710458472e+284
       x: [ 6.482e+50  1.111e+142]
     nit: 21
     jac: [ 0.000e+00 -2.222e+142]
    nfev: 63
    njev: 21